# Simulation of a corridor with different motivations

In this demonstration, we model a narrow corridor scenario featuring three distinct groups of agents. Among them, one group exhibits a higher level of motivation compared to the others.

We employ the collision-free speed model to determine the speed of each agent. This speed is influenced by the desired speed, denoted as $v^0$, the agent's radius $r$, and the slope factor $T$.

The varying motivation levels among the groups are represented by different $T$ values. The rationale for using $T$ to depict motivation is that highly motivated pedestrians, who are more aggressive in their movements, will quickly occupy any available space between them, correlating to a lower $T$ value. Conversely, less motivated pedestrians maintain a distance based on their walking speed, aligning with a higher $T$ value.

To accentuate this dynamic, the first group of agents will decelerate a few seconds into the simulation. As a result, we'll notice that the second group, driven by high motivation, will swiftly close distances and overtake the first group as it reduces speed. In contrast, the third group, with average motivation, will decelerate upon nearing the slower agents, without attempting to pass them. 

# Setting up the geometry

We will be using the a corridor 40 meters long and 4 meters wide.

In [ ]:
from shapely import Polygon
import pathlib
import pandas as pd
import numpy as np
import jupedsim as jps
import pedpy
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
print(f"JuPedSim: {jps.__version__}\nPedPy: {pedpy.__version__}")

In [ ]:
corridor = [(-1, -1), (60, -1), (60, 5), (-1, 5)]

areas = {}
areas["first"] = Polygon([[0, 0], [5, 0], [5, 4], [0, 4]])
areas["second"] = Polygon([[6, 0], [12, 0], [12, 4], [6, 4]])
areas["third"] = Polygon([[18, 0], [24, 0], [24, 4], [18, 4]])
areas["exit"] = Polygon([(56, 0), (59, 0), (59, 4), (56, 4)])

walkable_area = pedpy.WalkableArea(corridor)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1)
ax.set_aspect("equal")
_, ymin, _, ymax = walkable_area.bounds
ax.set_ylim(ymin - 2, ymax + 2)
pedpy.plot_walkable_area(walkable_area=walkable_area, axes=ax)
for name, area in areas.items():
    x, y = area.exterior.xy
    plt.fill(x, y, alpha=0.1)
    plt.plot(x, y, color="white")
    centroid = Polygon(area).centroid
    plt.text(
        centroid.x, centroid.y, name, ha="center", va="center", fontsize=8
    )

## Operational model
<a id="model"></a>
Now that the geometry is set, our subsequent task is to specify the model and its associated parameters.  
For this demonstration, we'll employ the "collision-free" model.  
However, since we are interested in two different motivation states, we will have to define two different time gaps.

In [ ]:
T_normal = 1.3
T_motivation = 0.1
v0_normal = 1.5
v0_slow = 0.5

Note, that in JuPedSim the model parameter $T$ is called `time_gap`. 

The values $1.3\, s$ and $0.1\, s$ are chosen according to the paper [Rzezonka2022, Fig.5](https://doi.org/10.1098/rsos.211822). 

## Setting Up the Simulation Object

Having established the model and geometry details, and combined with other parameters such as the time step $dt$, we can proceed to construct our simulation object as illustrated below:

In [ ]:
trajectory_file = "trajectories.sqlite"
simulation = jps.Simulation(
    dt=0.05,
    model=jps.CollisionFreeSpeedModel(
        strength_neighbor_repulsion=2.6,
        range_neighbor_repulsion=0.1,
        range_geometry_repulsion=0.05,
    ),
    geometry=walkable_area.polygon,
    trajectory_writer=jps.SqliteTrajectoryWriter(
        output_file=pathlib.Path(trajectory_file)
    ),
)

## Specifying Routing Details

At this point, we'll provide basic routing instructions, guiding the agents to progress towards an exit point, which is in this case at the end of the corridor.

In [ ]:
exit_id = simulation.add_exit_stage(areas["exit"])
journey_id = simulation.add_journey(jps.JourneyDescription([exit_id]))

## Defining and Distributing Agents

Now, we'll position the agents and establish their attributes, leveraging previously mentioned parameters.
We will distribute three different groups in three different areas.

- First area contains normally motivated agents. 
- The second area contains motivated agents that are more likely to close gaps to each other.
- The third area contains normally motivated agents. These agents will reduce their desired speeds after some seconds.

### Distribute normal agents in the first area

In [ ]:
total_agents_normal = 20
positions = jps.distribute_by_number(
    polygon=Polygon(areas["first"]),
    number_of_agents=total_agents_normal,
    distance_to_agents=0.4,
    distance_to_polygon=0.4,
    seed=45131502,
)

for position in positions:
    simulation.add_agent(
        jps.CollisionFreeSpeedModelAgentParameters(
            position=position,
            v0=v0_normal,
            time_gap=T_normal,
            journey_id=journey_id,
            stage_id=exit_id,
        )
    )

### Distribute motivated agents in the second area

In [ ]:
total_agents_motivated = 20
positions = jps.distribute_by_number(
    polygon=Polygon(areas["second"]),
    number_of_agents=total_agents_motivated,
    distance_to_agents=0.6,
    distance_to_polygon=0.6,
    seed=45131502,
)
for position in positions:
    simulation.add_agent(
        jps.CollisionFreeSpeedModelAgentParameters(
            position=position,
            v0=v0_normal,
            time_gap=T_motivation,
            journey_id=journey_id,
            stage_id=exit_id,
        )
    )

### Distribute normal agents in the third area

In [ ]:
total_agents_motivated_delay = 20
positions = jps.distribute_by_number(
    polygon=Polygon(areas["third"]),
    number_of_agents=total_agents_motivated_delay,
    distance_to_agents=0.8,
    distance_to_polygon=0.4,
    seed=45131502,
)
ids_third_group = set(
    [
        simulation.add_agent(
            jps.CollisionFreeSpeedModelAgentParameters(
                position=position,
                v0=v0_normal,
                time_gap=T_normal,
                journey_id=journey_id,
                stage_id=exit_id,
            )
        )
        for position in positions
    ]
)

## Executing the Simulation

With all components in place, we're set to initiate the simulation.
For this demonstration, the trajectories will be recorded in an sqlite database.

In [ ]:
while simulation.agent_count() > 0:
    simulation.iterate()
    if simulation.iteration_count() == 200:
        for id in ids_third_group:
            for agent in simulation.agents():
                if agent.id == id:
                    agent.model.v0 = v0_slow

## Visualizing the Trajectories

For trajectory visualization, we'll extract data from the sqlite database. A straightforward method for this is employing the jupedsim-visualizer.

In [ ]:
import plotly.graph_objects as go
import plotly.express as px
import sqlite3

DUMMY_SPEED = -1000


def read_sqlite_file(trajectory_file: str) -> pedpy.TrajectoryData:
    with sqlite3.connect(trajectory_file) as con:
        data = pd.read_sql_query(
            "select frame, id, pos_x as x, pos_y as y, ori_x as ox, ori_y as oy from trajectory_data",
            con,
        )
        fps = float(
            con.cursor()
            .execute("select value from metadata where key = 'fps'")
            .fetchone()[0]
        )
        walkable_area = (
            con.cursor().execute("select wkt from geometry").fetchone()[0]
        )
        return (
            pedpy.TrajectoryData(data=data, frame_rate=fps),
            pedpy.WalkableArea(walkable_area),
        )


def speed_to_color(speed, min_speed, max_speed):
    """Map a speed value to a color using a colormap."""
    normalized_speed = (speed - min_speed) / (max_speed - min_speed)
    r, g, b = plt.cm.jet_r(normalized_speed)[:3]
    return f"rgba({r*255:.0f}, {g*255:.0f}, {b*255:.0f}, 0.5)"


def get_line_color(disk_color):
    r, g, b, _ = [int(float(val)) for val in disk_color[5:-2].split(",")]
    brightness = (r * 299 + g * 587 + b * 114) / 1000
    return "black" if brightness > 127 else "white"


def create_orientation_line(row, line_length=0.2, color="black"):
    end_x = row["x"] + line_length * row["ox"]
    end_y = row["y"] + line_length * row["oy"]

    orientation_line = go.layout.Shape(
        type="line",
        x0=row["x"],
        y0=row["y"],
        x1=end_x,
        y1=end_y,
        line=dict(color=color, width=3),
    )
    return orientation_line


def get_geometry_traces(area):
    geometry_traces = []
    x, y = area.exterior.xy
    geometry_traces.append(
        go.Scatter(
            x=np.array(x),
            y=np.array(y),
            mode="lines",
            line={"color": "grey"},
            showlegend=False,
            name="Exterior",
            hoverinfo="name",
        )
    )
    for inner in area.interiors:
        xi, yi = zip(*inner.coords[:])
        geometry_traces.append(
            go.Scatter(
                x=np.array(xi),
                y=np.array(yi),
                mode="lines",
                line={"color": "grey"},
                showlegend=False,
                name="Obstacle",
                hoverinfo="name",
            )
        )
    return geometry_traces


def get_colormap(frame_data, max_speed):
    """Utilize scatter plots with varying colors for each agent instead of individual shapes.

    This trace is only to incorporate a colorbar in the plot.
    """
    scatter_trace = go.Scatter(
        x=frame_data["x"],
        y=frame_data["y"],
        mode="markers",
        marker=dict(
            size=frame_data["radius"] * 2,
            color=frame_data["speed"],
            colorscale="Jet_r",
            colorbar=dict(title="Speed [m/s]"),
            cmin=0,
            cmax=max_speed,
        ),
        text=frame_data["speed"],
        showlegend=False,
        hoverinfo="none",
    )

    return [scatter_trace]


def get_shapes_for_frame(frame_data, min_speed, max_speed):
    def create_shape(row):
        hover_trace = go.Scatter(
            x=[row["x"]],
            y=[row["y"]],
            text=[f"ID: {row['id']}, Pos({row['x']:.2f},{row['y']:.2f})"],
            mode="markers",
            marker=dict(size=1, opacity=1),
            hoverinfo="text",
            showlegend=False,
        )
        if row["speed"] == DUMMY_SPEED:
            dummy_trace = go.Scatter(
                x=[row["x"]],
                y=[row["y"]],
                mode="markers",
                marker=dict(size=1, opacity=0),
                hoverinfo="none",
                showlegend=False,
            )
            return (
                go.layout.Shape(
                    type="circle",
                    xref="x",
                    yref="y",
                    x0=row["x"] - row["radius"],
                    y0=row["y"] - row["radius"],
                    x1=row["x"] + row["radius"],
                    y1=row["y"] + row["radius"],
                    line=dict(width=0),
                    fillcolor="rgba(255,255,255,0)",  # Transparent fill
                ),
                dummy_trace,
                create_orientation_line(row, color="rgba(255,255,255,0)"),
            )
        color = speed_to_color(row["speed"], min_speed, max_speed)
        return (
            go.layout.Shape(
                type="circle",
                xref="x",
                yref="y",
                x0=row["x"] - row["radius"],
                y0=row["y"] - row["radius"],
                x1=row["x"] + row["radius"],
                y1=row["y"] + row["radius"],
                line_color=color,
                fillcolor=color,
            ),
            hover_trace,
            create_orientation_line(row, color=get_line_color(color)),
        )

    results = frame_data.apply(create_shape, axis=1).tolist()
    shapes = [res[0] for res in results]
    hover_traces = [res[1] for res in results]
    arrows = [res[2] for res in results]
    return shapes, hover_traces, arrows


def create_fig(
    initial_agent_count,
    initial_shapes,
    initial_arrows,
    initial_hover_trace,
    initial_scatter_trace,
    geometry_traces,
    frames,
    steps,
    area_bounds,
    width=800,
    height=800,
    title_note: str = "",
):
    """Creates a Plotly figure with animation capabilities.

    Returns:
        go.Figure: A Plotly figure with animation capabilities.
    """

    minx, miny, maxx, maxy = area_bounds
    title = f"<b>{title_note + '  |  ' if title_note else ''}Number of Agents: {initial_agent_count}</b>"
    fig = go.Figure(
        data=geometry_traces + initial_scatter_trace
        # + hover_traces
        + initial_hover_trace,
        frames=frames,
        layout=go.Layout(
            shapes=initial_shapes + initial_arrows, title=title, title_x=0.5
        ),
    )
    fig.update_layout(
        updatemenus=[_get_animation_controls()],
        sliders=[_get_slider_controls(steps)],
        autosize=False,
        width=width,
        height=height,
        xaxis=dict(range=[minx - 0.5, maxx + 0.5]),
        yaxis=dict(
            scaleanchor="x", scaleratio=1, range=[miny - 0.5, maxy + 0.5]
        ),
    )

    return fig


def _get_animation_controls():
    """Returns the animation control buttons for the figure."""
    return {
        "buttons": [
            {
                "args": [
                    None,
                    {
                        "frame": {"duration": 100, "redraw": True},
                        "fromcurrent": True,
                    },
                ],
                "label": "Play",
                "method": "animate",
            },
        ],
        "direction": "left",
        "pad": {"r": 10, "t": 87},
        "showactive": False,
        "type": "buttons",
        "x": 0.1,
        "xanchor": "right",
        "y": 0,
        "yanchor": "top",
    }


def _get_slider_controls(steps):
    """Returns the slider controls for the figure."""
    return {
        "active": 0,
        "yanchor": "top",
        "xanchor": "left",
        "currentvalue": {
            "font": {"size": 20},
            "prefix": "Frame:",
            "visible": True,
            "xanchor": "right",
        },
        "transition": {"duration": 100, "easing": "cubic-in-out"},
        "pad": {"b": 10, "t": 50},
        "len": 0.9,
        "x": 0.1,
        "y": 0,
        "steps": steps,
    }


def _get_processed_frame_data(data_df, frame_num, max_agents):
    """Process frame data and ensure it matches the maximum agent count."""
    frame_data = data_df[data_df["frame"] == frame_num]
    agent_count = len(frame_data)
    dummy_agent_data = {"x": 0, "y": 0, "radius": 0, "speed": DUMMY_SPEED}
    while len(frame_data) < max_agents:
        dummy_df = pd.DataFrame([dummy_agent_data])
        frame_data = pd.concat([frame_data, dummy_df], ignore_index=True)
    return frame_data, agent_count


def animate(
    data: pedpy.TrajectoryData,
    area: pedpy.WalkableArea,
    *,
    every_nth_frame: int = 50,
    width: int = 800,
    height: int = 800,
    radius: float = 0.2,
    title_note: str = "",
):
    data_df = pedpy.compute_individual_speed(
        traj_data=data,
        frame_step=5,
        speed_calculation=pedpy.SpeedCalculation.BORDER_SINGLE_SIDED,
    )
    data_df = data_df.merge(data.data, on=["id", "frame"], how="left")
    data_df["radius"] = radius
    min_speed = data_df["speed"].min()
    max_speed = data_df["speed"].max()
    max_agents = data_df.groupby("frame").size().max()
    frames = []
    steps = []
    unique_frames = data_df["frame"].unique()
    selected_frames = unique_frames[::every_nth_frame]
    geometry_traces = get_geometry_traces(area.polygon)
    initial_frame_data = data_df[data_df["frame"] == data_df["frame"].min()]
    initial_agent_count = len(initial_frame_data)
    initial_shapes, initial_hover_trace, initial_arrows = get_shapes_for_frame(
        initial_frame_data, min_speed, max_speed
    )
    color_map_trace = get_colormap(initial_frame_data, max_speed)
    for frame_num in selected_frames:
        frame_data, agent_count = _get_processed_frame_data(
            data_df, frame_num, max_agents
        )
        shapes, hover_traces, arrows = get_shapes_for_frame(
            frame_data, min_speed, max_speed
        )
        title = f"<b>{title_note + '  |  ' if title_note else ''}Number of Agents: {agent_count}</b>"
        frame = go.Frame(
            data=geometry_traces + hover_traces,
            name=str(frame_num),
            layout=go.Layout(
                shapes=shapes + arrows,
                title=title,
                title_x=0.5,
            ),
        )
        frames.append(frame)

        step = {
            "args": [
                [str(frame_num)],
                {
                    "frame": {"duration": 100, "redraw": True},
                    "mode": "immediate",
                    "transition": {"duration": 500},
                },
            ],
            "label": str(frame_num),
            "method": "animate",
        }
        steps.append(step)

    return create_fig(
        initial_agent_count,
        initial_shapes,
        initial_arrows,
        initial_hover_trace,
        color_map_trace,
        geometry_traces,
        frames,
        steps,
        area.bounds,
        width=width,
        height=height,
        title_note=title_note,
    )

In [ ]:
agent_trajectories, walkable_area = read_sqlite_file(trajectory_file)
animate(
    agent_trajectories,
    walkable_area,
    every_nth_frame=5,
    width=1000,
    height=500,
)

## Notes and Comments

It's noticeable that members of the second group tend to draw nearer to each other compared to those in the first group, primarily attributed to their lower $T$ values. As the third group begins to decelerate after a while, due to an adjustment in the target speed $v_0$, the second group seizes this opportunity to bridge the distance and surpass them. 

Conversely, the first group maintains a consistent pace and doesn't attempt to overtake the now-lagging third group.